# Introduction

This is a comparison scratch pad between a generator-based
and prototype-based application ui

The overall goal is to split my application into a "command"
layer, that is logic dense but pure, and a "runtime" layer
that is devoid of business-specific logic but can run effects

In other words, the elm architecture

# Generators
A generator api models the application as a pausable function
that can "call out" to the runtime with a command represented as
data, get a response back, and continue executing.

Modules can be built on top of the command api to perform complex
business logic in a largely effect-less way

This is akin to a monad-like pattern, as it lets me write logic in
an instruction-like imperative style, while leaving effectful interpretation up to an outside interpreter

The runtime defines what commands it can accept. If it can't
understand some command, it can trigger an exception within the
function

Although a generator is itself an object, it isn't written like one,
resulting in a more function-oriented style

Because the application logic "yields" to the runtime, I can determine
at runtime how each command is implemented, whether it runs an actual
effect or just collects the commands for analysis

## Backend

### Interpreter

First, we define a `perform` function that acts as our top-level operation on the "application" data structure

In [ ]:
async function perform(commandset: Generator<any>) {
  // Read each command
  for (const command of commandset) {
    // Perform different actions based on the command
    switch(command.type) {
      case 'SQL': {
        // Perform an actual effect
        const result = await sql.exec(command.query)

        // Send the result back into the application, so
        // it can know what the next instruction is
        commandset.next(result)
      }
    }
  }
}

### Fake Interpreter

Because we take the entire application as input, we are free to define alternative interpreters

For example, an interpreter that only performs in-memory operations

In [1]:
// This is a testing harness to run the application without using
// real effects
async function fakePerform(commandset: Generator<any>) {
  // Read each command
  for (const command of commandset) {
    // Perform different actions based on the command
    switch(command.type) {
      case 'SQL': {
        // Log the query for testing
        console.log(command.query)

        // Perform a FAKE effect that's just an empty result
        const result = await Promise.resolve([])

        // Send the result back into the application, so
        // it can know what the next instruction is
        commandset.next(result)
      }
    }
  }
}

## Frontend

### Command Setup

Next, we define the command API of our system.

This is mostly just some plumbing work to define our command objects

In [ ]:
// This is a helper to build commands
const App = {
  sql(query: string) {
    return {
      type: 'SQL',
      query
    }
  }
}

// This is an example module that is command-aware
const Insights = {
  query(template: string) {
    // This just builds a command to send to the runtime.
    // This could also itself be a generator if we wanted
    // to transform the data after getting a response
    return App.sql(template)
  }
}

const sql = (str: any, ...args: any) => str

### Define our business logic

With our command protocol defined, we can write some business logic scripts

These are defined as generator functions that `yield` commands structured in the way we defined above

In [1]:
// This is an example application commandset
function* getSite() {
  // Yield a command. The application determines how this
  // will be interpreted
  const site = yield Insights.query(sql`
    SELECT
      COUNT(*) as total,
      COUNT(DISTINCT fingerprint) as unique,
      created_at as date
    GROUP BY
      date(created_at)
  `)

  // Return data back to the runtime
  return {
    site
  }
}

TypeError: sql.exec is not a function

## Combine operations at the top-level

With our core logic and interpreters defined, we can now perform some top-level effectful operations

It may be tempting to wrap this up as a single module, but the point is that we only link the effectful
interpreter with the pure logic engine at the very highest-point

In [2]:
// Let the app interpret the commandset data structure
await perform(getSite())

// Run the commandset without actually running effects
await fakePerform(getSite())

// You can still run Insights logic independently of the rest
// of the application
const _command = Insights.query(sql`SELECT *`)

ReferenceError: perform is not defined

# Prototypes

A prototype api models the application as a stateful _object_ that
can "call out" to it's runtime prototype via a super call.

Modules that know the application method protocol can be built on top
to perform complex business logic, and be "mixed-in" to the application
server

This is a more javascript-native pattern

Because the application calls out to the runtime via super, I can replace
the prototype implementation at runtime to either an actual affect, or
just collect and inspect calls


In [4]:
// This is the effect-ful runtime server
// This is the "backend" of the application.
// 
// It doesn't know anything about the application business logic, just
// how to translate general messages
const runtime = {
  async sql(query) {
    return await sql.exec(query)
  }
}


// This is a fake server we can inject when testing
const fakeRuntime = {
  async sql(query) {
    // Log the query and just return an empty result
    console.log(query)
    return await Promise.resolve([])
  }
}


// --- SERVICE BOUNDARY ---


// This is an example module that calls out to the runtime
//
// This module is mostly unaware of the shenanigans we're doing
// with prototypes. It calls out to the code it needs, same as always.
//
// This ensures it can be used or tested in isolation like a normal
// library
const insights = {
  async query(template: string) {
    // This just sends a message to send to the runtime.
    // This works just like a regular function call so
    // we can do any extra logic or calls we want
    return await runtime.sql(template)
  }
}

// This is an example application function
async function getSite() {
  // Call an effectful function like normal
  //
  // Notice how we need to use a bound Insights module here to
  // ensure it has access to the appropriate runtime methods
  // 
  // We will dynamically determine how this will be interpreted
  const site = await insights.query(sql`
    SELECT
      COUNT(*) as total,
      COUNT(DISTINCT fingerprint) as unique,
      created_at as date
    GROUP BY
      date(created_at)
  `)

  // Return data back to the runtime
  return {
    site
  }
}


// Call the effectful api
await app.getSite()

// Sub in an effect-less api
Object.setPrototypeOf(app, fakeRuntime)

await app.getSite()

ReferenceError: app is not defined

This is a mixed approach that uses a generator-based api on the frontend, and
a prototype-based api for the backend server implementation

I prefer the generator approach for the frontend because it doesn't
require any weird `this` finagling or object re-assignment.

I prefer the prototype approach for the backend server because the
function interpreter is a bit unwieldy to implement functionally. I'd rather
implement it as a dynamic, message driven object like Elixir. I'm
inherently dealing with a stateful server on this end

In [3]:
// This is a base interpreter server mixin.
// By default it only understands the `perform` message.
//
// For each command, it'll try and find an associated message
// no the object using the mixin
const baseRuntime = {
  // This function interprets the application into real effects
  perform(commandset: Generator<any>) {
    // Read each command
    for (const command of commandset) {
      // Perform different actions based on the command
      const result = this[command.type](command)

      if (result instanceof Promise) {
        result
          .then(resolved => commandset.next(resolved))
          .catch(rejected => commandset.throw(rejected))
      }

      // Send the result back into the application, so
      // it can know what the next instruction is
      commandset.next(result)
    }
  }
}

// Create a our app-specific runtime, which inherits from the base
// runtime.
// 
// From here we can add add dynamic handler functions without affecting
// the base. 
const runtime = Object.create(baseRuntime)

// Here we dynamically create some command interpreters.
// This is a better way to resolve (type -> handler) than
// a massive switch statement. It's more elixir-like I think
runtime['SQL'] = async () => {
  return await sql.exec(command.query)
}

// This is a testing runtime to run the application without using
// real effects
const fakeRuntime = Object.create(baseRuntime)

// Similar to above, we add command handlers
fakeRuntime['SQL'] = async () => {
  // Log the query for testing
  console.log(command.query)

  // Perform a FAKE effect that's just an empty result
  return await Promise.resolve([])
}

// --- SERVICE BOUNDARY ---

// This is a helper to build commands
//
// The commands defined here are completely disconnected
// from the commands implemented on the interpreter side
//
// Similar to HTML, the onus is on the interpreter to understand
// or safely disregard a command. The interpreter is adaptable and
// enhancement-based
const App = {
  sql(query: string) {
    return {
      type: 'SQL',
      query
    }
  }
}

// This is a module that knows about the
// runtime data protocol
const Insights = {
  query(template: string) {
    // This just builds a command to send to the runtime.
    // This could also itself be a generator if we wanted
    // to transform the data after getting a response
    return App.sql(template)
  }
}

const sql = (str: any, ...args: any) => str

// This is an example application commandlist
function* getSite() {
  // Yield a command. The application determines how this
  // will be interpreted
  const site = yield Insights.query(sql`
    SELECT
      COUNT(*) as total,
      COUNT(DISTINCT fingerprint) as unique,
      created_at as date
    GROUP BY
      date(created_at)
  `)

  // Return data back to the runtime
  return {
    site
  }
}

// Let the runtime interpret the commandlist data structure
await runtime.perform(getSite())

// Interpret the commandlist without actually running effects
await fakeRuntime.perform(getSite())

// You can still run Insights logic independently of the rest
// of the application
Insights.query(sql`SELECT *`) // -> { type: 'SQL', ... }

// You can also run commandlists like regular iterables for ad-hoc
// inspection
for (const command of getSite()) {
  console.log(command)
}


{
  type: "SQL",
  query: [
    "\n" +
      "    SELECT\n" +
      "      COUNT(*) as total,\n" +
      "      COUNT(DISTINCT fingerprint) as unique,\n" +
      "      created_at as date\n" +
      "    GROUP BY\n" +
      "      date(created_at)\n" +
      "  "
  ]
}


### FUTURE WORK

Middleware, how does that work in redux?

It's logic we can put between calling a command and sending it to
the runtime, in a generalized way that intercepts ALL commands.

I don't think this is too useful for my cases. Only real use I can
think of would be command-side escape hatches, such as logging/reporting

I could do a "wrapping" pattern similar to Elm, like how I wrapped a
websocket receiver/handler around my application handler. I don't really
have an app state though, so perhaps this isn't strictly applicable here.
It _could_ be if I passed information in, such as a request object, but
that's already taken care of for my by the application router